In [1]:
import pandas as pd
import scipy as sp
import numpy as np
from matplotlib import pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from scipy import stats
import re
plt.style.use('ggplot')

In [2]:
t10 = pd.read_csv("t10.csv")

In [128]:
made_lst = list(t10.groupby("made").groups.keys())

In [4]:
t10.head()

,cty,drive,eng,exter,fuel,hwy,inter,made,mileage,model,price,slreview,slrzip,title,tran,year,age
0,27.0,FWD,Gas I4 1.8L/110,Silver,Gasoline,35.0,NaN,Toyota,80135,Corolla,6990,2.1,7834.0,Used 2009 Toyota Corolla LE,Automatic,2009,10
1,24.0,FWD,Gas I4 2.4L/146,Red,Gasoline,33.0,NaN,Toyota,157470,Camry,4390,2.1,7834.0,Used 2006 Toyota Camry LE,5-Speed Automatic w/OD,2006,13
2,23.0,AWD,2.5L 4-Cylinder DOHC Dual VVT-i,Silver,Gasoline,30.0,Ash,Toyota,12001,RAV4,28997,4.6,6807.0,Used 2018 Toyota RAV4 XLE,6-Speed Automatic,2018,1
3,17.0,AWD,4.0L V6 SMPI DOHC,Silver,Gasoline,21.0,Bl Gr Sft Syn Lth,Toyota,5,4Runner,37639,4.6,6807.0,Used 2018 Toyota 4Runner SR5 Premium,5-Speed Automatic with Overdrive,2018,1
4,13.0,AWD,5.7L V8 DOHC Dual VVT-i 32V,Black,Gasoline,18.0,Terra Semi-Aniline Perfor,Toyota,201,Land,80997,4.6,6807.0,Used 2019 Toyota Land Cruiser Base,8-Speed Automatic (ECT-i),2019,0


In [95]:
def getdf (brand):
    e = t10.loc[t10['made'] == brand][['age', 'mileage', 'price', 'model']]
    e = e[e.groupby("age").price.transform(lambda x: np.abs(x-x.mean())/x.std() < 3)]
    e = e[e.groupby("age").mileage.transform(lambda x: np.abs(x-x.mean())/x.std() < 3)]
    e = pd.get_dummies(e)
    return e

In [97]:
BMW = getdf("BMW")

In [93]:
def spratedata(df):
    X = df.drop('price', axis = 1)
    X = X.assign(agesqrt = X.age**2)
    y = df['price']
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)
    return X_train, X_test, y_train, y_test

In [99]:
BMW.head()

,age,mileage,price,model_128,model_135,model_228,model_230,model_3,model_320,model_320i,...,model_M6,model_X1,model_X2,model_X3,model_X4,model_X5,model_X6,model_Z4,model_i,model_i3
242,10,91877,11634,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
243,5,90641,22000,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
244,4,66927,22000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
245,3,45854,31995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
246,3,32758,31995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [107]:
BMW_X = BMW.drop('price', axis = 1)
BMW_X = BMW_X.assign(agesqrt = BMW_X.age**2)
BMW_y = BMW['price']
BMW_X_train, BMW_X_test, BMW_y_train, BMW_y_test = train_test_split(BMW_X,BMW_y,test_size=0.2, random_state=0)

In [108]:
lm = linear_model.LinearRegression()
BMW_model = lm.fit(BMW_X_train,BMW_y_train)

In [109]:
BMW_model.score(BMW_X_test, BMW_y_test)

0.8779327787288941

In [114]:
BMW_model.coef_[0]+BMW_model.coef_[-1]

-5506.129471328685

In [119]:
def getmodel (brand):
    e = t10.loc[t10['made'] == brand][['age', 'mileage', 'price', 'model']]
    e = e[e.groupby("age").price.transform(lambda x: np.abs(x-x.mean())/x.std() < 3)]
    e = e[e.groupby("age").mileage.transform(lambda x: np.abs(x-x.mean())/x.std() < 3)]
    e = pd.get_dummies(e)
    X = e.drop("price", axis = 1)
    X = X.assign(agesqrt = X.age**2)
    y = e['price']
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)
    lm = linear_model.LinearRegression()
    model = lm.fit(X_train, y_train)
    return model.score(X_test, y_test), model.coef_[0] + model.coef_[-1]

In [121]:
def getmodel_woagesqrt (brand):
    e = t10.loc[t10['made'] == brand][['age', 'mileage', 'price', 'model']]
    e = e[e.groupby("age").price.transform(lambda x: np.abs(x-x.mean())/x.std() < 3)]
    e = e[e.groupby("age").mileage.transform(lambda x: np.abs(x-x.mean())/x.std() < 3)]
    e = pd.get_dummies(e)
    X = e.drop("price", axis = 1)
    y = e['price']
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)
    lm = linear_model.LinearRegression()
    model = lm.fit(X_train, y_train)
    return model.score(X_test, y_test), model.coef_[0]

In [131]:
getmodel("")

(0.8320252599588952, -2152.134460701011)

In [132]:
getmodel_woagesqrt("Honda")

(0.8205351173965031, -953.0238556925322)

In [129]:
map()

['BMW',
 'Chevrolet',
 'Ford',
 'Honda',
 'Hyundai',
 'Jeep',
 'Lexus',
 'Mercedes-Benz',
 'Nissan',
 'Toyota']

In [140]:
for i, e in enumerate(made_lst):
    print(made_lst[i]+str(getmodel(e)))

BMW(0.8779327787288941, -5506.129471328685)
Chevrolet(0.8087948585336119, -1694.2070082593234)
Ford(0.7820649872097638, -1556.1679095654738)
Honda(0.8320252599588952, -2152.134460701011)
Hyundai(0.7522954272352842, -421.1348448537532)
Jeep(0.7600589669544168, -1481.8788394055669)
Lexus(0.8591510251217482, -4557.472701693688)
Mercedes-Benz(0.8485240626887398, -5176.816151883961)
Nissan(0.8100503636807886, -1170.0731448444187)
Toyota(0.8103637859526494, -1194.7748747648325)


In [141]:
for i, e in enumerate(made_lst):
    print(made_lst[i]+str(getmodel_woagesqrt(e)))

BMW(0.8179163748602593, -2109.7389630910075)
Chevrolet(0.8039174136910995, -1029.163202835425)
Ford(0.7757884141782474, -1032.3854194844164)
Honda(0.8205351173965031, -953.0238556925322)
Hyundai(0.7537644549310939, -511.5661339795682)
Jeep(0.7593300498601839, -982.9621808296453)
Lexus(0.7856918510158866, -1705.488937066368)
Mercedes-Benz(0.8223930183945802, -2453.8225735880096)
Nissan(0.8066091729467673, -702.0867731348707)
Toyota(0.8095084276533652, -888.9396285301241)
